In [102]:
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage
from langchain_core.tracers.context import tracing_v2_enabled

# Set up the model with generation params
model = ChatOllama(
    model="gemma3:4b",
    temperature=0.7,
)

# Wrap with LangSmith tracing
with tracing_v2_enabled(project_name="default"):
    response = model.invoke([
        HumanMessage(content="What's something cool about ancient Egypt, only say 1 short thing!")
    ])
    print(response.content)


Ancient Egyptians used makeup – including kohl eyeliner – not just for beauty, but to ward off evil spirits and protect their eyes!


In [104]:
for chunk in model.stream([
    HumanMessage(content="Tell me a short poem about the stars.")
]):
    print(chunk.content, end="", flush=True)


In velvet skies, a silent grace,
The stars ignite, a timeless space.
Tiny diamonds, scattered bright,
Whispering secrets in the night.

Each distant sun, a fiery plea,
Lost in the vast immensity.
A cosmic dance, serene and slow,
Where stardust dreams begin to grow.

In [105]:
messages = [
    [HumanMessage(content="Who built the pyramids?")],
    [HumanMessage(content="What is the speed of light?")],
]

responses = model.batch(messages)
for res in responses:
    print(res.content)


That's a fantastic and surprisingly complex question! The short answer is: **the ancient Egyptians built the pyramids.** However, the *how* and *why* are far more nuanced than a simple statement. Here’s a breakdown of what we know, and what’s still debated:

**1. The Workers – It Wasn't Just Slaves!**

* **Skilled Labor:** Contrary to popular belief, the pyramids were *not* built primarily by slaves. Extensive archaeological evidence shows that the workforce consisted of paid laborers – skilled craftspeople, engineers, architects, stonemasons, and laborers. They were organized into crews, and many had housing, food, and even medical care.
* **Organized Crews:** These workers were organized into teams, often with names reflecting their skills or origins (e.g., the "Red Crown" crew).
* **Seasonal Labor:** It’s believed that much of the work was done during the annual Nile flood – when the river was at its highest, the fields were underwater, and agricultural work was impossible. This pro

In [107]:
from langchain_core.messages import ToolMessage
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.schema import HumanMessage

@tool 
def get_weather(city: str) -> str:
    """Tool for getting the weather, Get the current weather for a city."""
    return f"The weather in {city} sunny!"

@tool 
def get_name() -> str:
    """Tool for getting the name, Get the name."""
    return f"The name is Youssef Malek!"

model = ChatOllama(
    model="llama3.2",
    temperature=0.7, 
    num_predict=200,
).bind_tools([get_weather,get_name])

# List of conversations (1 per query)
message_batches = [
    [HumanMessage(content="What is the weather in Cairo?")],
    [HumanMessage(content="What is the name?")]
]

# First call: send both queries as a batch
ai_responses = model.batch(message_batches)

# Process tool calls and build final batches
final_batches = []

for i, ai_msg in enumerate(ai_responses):
    messages = message_batches[i] + [ai_msg]
    
    # If the model called a tool...
    if ai_msg.tool_calls:
        for tool_call in ai_msg.tool_calls:
            selected_tool = {
                "get_weather": get_weather,
                "get_name": get_name
            }[tool_call["name"].lower()]
            tool_output = selected_tool.invoke(tool_call["args"])
            messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    
    final_batches.append(messages)

# Final call with the tool outputs included
final_responses = model.batch(final_batches)

# Print final responses
for res in final_responses:
    print(res.content)
    print("\n")


According to the current conditions, the temperature in Cairo is 28°C (82°F), with plenty of sunshine and clear skies. It's a great day to explore the Pyramids or take a stroll along the Nile River. Would you like more information about the weather forecast for Cairo?


It looks like I don't have any information about a person named Youssef Malek. The tool call response didn't provide enough context for me to give an accurate answer.

Can you please provide more information or clarify the question? I'll do my best to help!

If you'd like, I can try again with a new tool call. Just let me know what kind of information you're looking for!




In [95]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])